# Overview

This notebook contains my solutions for **<a href="https://adventofcode.com/2023" target="_blank">Advent of Code 2023</a>**.

A few notes...
- The source for this notebook source lives in my GitHub repo, <a href="https://github.com/derailed-dash/Advent-of-Code/blob/master/src/AoC_2023/Dazbo's_Advent_of_Code_2023.ipynb" target="_blank">here</a>.
- You can run this Notebook wherever you like. For example, you could...
  - Run it locally, in your own Jupyter environment.
  - Run it in a cloud-based Jupyter environment, with no setup required on your part!  For example, <a href="https://colab.research.google.com/github/derailed-dash/Advent-of-Code/blob/master/src/AoC_2023/Dazbo's_Advent_of_Code_2023.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colab"/></a>
- **To run the notebook, execute the cells in the [Setup](#Setup) section, as described below. Then you can run the code for any given day.**
- Be mindful that the first time you run this notebook, you will need to **obtain your AoC session key** and store it, if you have not done so already. This allows the notebook to automatically retrieve your input data. (See the guidance in the **[Get Access to Your AoC Data](#Get-Access-to-Your-AoC-Data)** section for details.)
- Use the navigation menu on the left to jump to any particular day.
- All of my AoC solutions are documented in my <a href="https://aoc.just2good.co.uk/" target="_blank">AoC Python Walkthrough site</a>.

# Setup

You need to run all cells in this section, before running any particular day solution.

## Packages and Imports

Here we use `pip` to install the packages used by my solutions in this event.

In [1]:
%pip install jupyterlab-lsp colorama python-dotenv ipykernel 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations
from dataclasses import asdict, dataclass, field
from enum import Enum, auto
from functools import cache, reduce
from itertools import permutations, combinations, count
from collections import Counter, deque, defaultdict
import heapq
import copy
import operator
import logging
import time
import os
import re
import ast
import unittest
import requests
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv
from pathlib import Path
from getpass import getpass
from colorama import Fore
from IPython.display import display
from IPython.core.display import Markdown

## Logging and Output

Set up a new logger that uses `ColouredFormatter`, such that we have coloured logging.  The log colour depends on the logging level.

In [3]:
##########################################################################
# SETUP LOGGING
#
# Create a new instance of "logger" in the client application
# Set to your preferred logging level
# And add the stream_handler from this module, if you want coloured output
##########################################################################

# logger for aoc_commons only
logger = logging.getLogger(__name__) # aoc_common.aoc_commons
logger.setLevel(logging.INFO)
stream_handler = None

class ColouredFormatter(logging.Formatter):
    """ Custom Formater which adds colour to output, based on logging level """

    level_mapping = {"DEBUG": (Fore.BLUE, "DBG"),
                     "INFO": (Fore.GREEN, "INF"),
                     "WARNING": (Fore.YELLOW, "WRN"),
                     "ERROR": (Fore.RED, "ERR"),
                     "CRITICAL": (Fore.MAGENTA, "CRT")
    }

    def __init__(self, *args, apply_colour=True, shorten_lvl=True, **kwargs) -> None:
        """ Args:
            apply_colour (bool, optional): Apply colouring to messages. Defaults to True.
            shorten_lvl (bool, optional): Shorten level names to 3 chars. Defaults to True.
        """
        super().__init__(*args, **kwargs)
        self._apply_colour = apply_colour
        self._shorten_lvl = shorten_lvl

    def format(self, record):
        if record.levelname in ColouredFormatter.level_mapping:
            new_rec = copy.copy(record)
            colour, new_level = ColouredFormatter.level_mapping[record.levelname]

            if self._shorten_lvl:
                new_rec.levelname = new_level

            if self._apply_colour:
                msg = colour + super().format(new_rec) + Fore.RESET
            else:
                msg = super().format(new_rec)

            return msg

        # If our logging message is not using one of these levels...
        return super().format(record)

if not stream_handler:
    stream_handler = logging.StreamHandler()
    stream_fmt = ColouredFormatter(fmt='%(asctime)s.%(msecs)03d:%(name)s - %(levelname)s: %(message)s',
                                   datefmt='%H:%M:%S')
    stream_handler.setFormatter(stream_fmt)
    
if not logger.handlers:
    # Add our ColouredFormatter as the default console logging
    logger.addHandler(stream_handler)

def retrieve_console_logger(script_name):
    """ Create and return a new logger, named after the script
    So, in your calling code, add a line like this:
    logger = ac.retrieve_console_logger(locations.script_name)
    """
    a_logger = logging.getLogger(script_name)
    a_logger.addHandler(stream_handler)
    a_logger.propagate = False
    return a_logger

def setup_file_logging(a_logger: logging.Logger, folder: str|Path=""):
    """ Add a FileHandler to the specified logger. File name is based on the logger name.
    In calling code, we can add a line like this:
    td.setup_file_logging(logger, locations.output_dir)

    Args:
        a_logger (Logger): The existing logger
        folder (str): Where the log file will be created. Will be created if it doesn't exist
    """
    Path(folder).mkdir(parents=True, exist_ok=True)     # Create directory if it does not exist
    file_handler = logging.FileHandler(Path(folder, a_logger.name + ".log"), mode='w')
    file_fmt = logging.Formatter(fmt="%(asctime)s.%(msecs)03d:%(name)s:%(levelname)8s: %(message)s",
                                datefmt='%H:%M:%S')
    file_handler.setFormatter(file_fmt)
    a_logger.addHandler(file_handler)

In [4]:
def top_and_tail(data, block_size=5, include_line_numbers=True, zero_indexed=False):
    """ Print a summary of a large amount of data 

    Args:
        data (_type_): The data to present in summary form.
        block_size (int, optional): How many rows to include in the top, and in the tail.
        include_line_numbers (bool, optional): Prefix with line number. Defaults to True.
        zero_indexed (bool, optional): Lines start at 0? Defaults to False.
    """
    if isinstance(data, list):
        # Get the number of digits of the last item for proper alignment
        num_digits_last_item = len(str(len(data)))

        # Format the string with line number
        def format_with_line_number(idx, line):
            start = 0 if zero_indexed else 1
            if include_line_numbers:
                return f"{idx + start:>{num_digits_last_item}}: {line}"
            else:
                return line

        start = 0 if zero_indexed else 1
        if len(data) < 11:
            return "\n".join(format_with_line_number(i, line) for i, line in enumerate(data))
        else:
            top = [format_with_line_number(i, line) for i, line in enumerate(data[:block_size])]
            tail = [format_with_line_number(i, line) for i, line in enumerate(data[-block_size:], start=len(data)-block_size)]
            return "\n".join(top + ["..."] + tail)
    else:
        return data

## Get Access to Your AoC Data

Now provide your unique AoC session key, in order to download your input data. You can get this by:
1. Logging into [Advent of Code](https://adventofcode.com/).
1. From your browser, open Developer Tools. (In Chrome, you can do this by pressing F12.)
1. Open the `Application` tab.
1. Storage -> Cookies -> https://adventofcode.com
1. Copy the value associated with the cookie called `session`.
1. Once you've determiend your session key, I recommend you store it in a file called `.env`, in your `Advent-of-Code` folder, like this: \
`AOC_SESSION_COOKIE=536...your-own-session-key...658` \
This notebook will try to retrieve the key from that location.  If it is unable to retrieve the key, it will prompt you to enter your key in the cell below.

![Finding the session cookie](https://aoc.just2good.co.uk/assets/images/aoc-cookie.png)



In [5]:
def get_envs_from_file() -> bool:
    """ Look for .env files, read variables from it, and store as environment variables """
    potential_path = ".env"
    for _ in range(3):
        logger.debug("Trying .env at %s", os.path.realpath(potential_path))
        if os.path.exists(potential_path):
            logger.info("Using .env at %s", os.path.realpath(potential_path))
            load_dotenv(potential_path, verbose=True)
            return True
        
        potential_path = os.path.join('..', potential_path)
   
    logger.warning("No .env file found.")
    return False

get_envs_from_file() # read env variables from a .env file, if we can find one

20:38:58.133:__main__ - INF: Using .env at F:\Users\Darren\localdev\Python\Advent-of-Code\.env


True

In [6]:
if os.getenv('AOC_SESSION_COOKIE'):
    logger.info('Session cookie retrieved: %s...%s', os.environ['AOC_SESSION_COOKIE'][0:6], os.environ['AOC_SESSION_COOKIE'][-6:])
else: # it's not in our environment variables, so we'll need to input the value
    os.environ['AOC_SESSION_COOKIE'] = getpass('Enter AoC session key: ')

20:38:58.148:__main__ - INF: Session cookie retrieved: 53616c...13d33c


## Load Helpers and Useful Classes

Now we load a bunch of helper functions and classes.

### Locations

Where any input and output files get stored.

<img src="https://aoc.just2good.co.uk/assets/images/notebook-content-screenshot.png" width="320" />


In [7]:
#################################################################
# Paths and Locations
#################################################################

@dataclass
class Locations:
    """ Dataclass for storing various location properties """
    script_name: str
    script_dir: Path
    input_dir: Path
    output_dir: Path
    input_file: Path

def get_locations(script_name, folder="") -> Locations:
    """ Set various paths, based on the location of the calling script. """
    current_directory = os.getcwd()
    script_dir = Path(Path().resolve(), folder, script_name)
    input_dir = Path(script_dir, "input")
    output_dir = Path(script_dir, "output")
    input_file = Path(input_dir, "input.txt")

    return Locations(script_name, script_dir,
                     input_dir,
                     output_dir,
                     input_file)

### Retrieve the Input Data

This works by using your unique session cookie to retrieve your input data. E.g. from a URL like:

`https://adventofcode.com/2015/day/1/input`

In [8]:
##################################################################
# Retrieving input data
##################################################################

def write_puzzle_input_file(year: int, day, locations: Locations):
    """ Use session key to obtain user's unique data for this year and day.
    Only retrieve if the input file does not already exist.
    Return True if successful.
    Requires env: AOC_SESSION_COOKIE, which can be set from the .env.
    """
    if os.path.exists(locations.input_file):
        logger.debug("%s already exists", os.path.basename(locations.input_file))
        return os.path.basename(locations.input_file)

    session_cookie = os.getenv('AOC_SESSION_COOKIE')
    if not session_cookie:
        raise ValueError("Could not retrieve session cookie.")

    logger.info('Session cookie retrieved: %s...%s', session_cookie[0:6], session_cookie[-6:])

    # Create input folder, if it doesn't exist
    if not locations.input_dir.exists():
        locations.input_dir.mkdir(parents=True, exist_ok=True)

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    
    # Don't think we need to set a user-agent
    # headers = {
    #     "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    # }
    cookies = { 
        "session": session_cookie
    }
    response = requests.get(url, cookies=cookies, timeout=5)

    data = ""
    if response.status_code == 200:
        data = response.text

        with open(locations.input_file, 'w') as file:
            logger.debug("Writing input file %s", os.path.basename(locations.input_file))
            file.write(data)
            return data
    else:
        raise ValueError(f"Unable to retrieve input data.\n" +
                         f"HTTP response: {response.status_code}\n" +
                         f"{response.reason}: {response.content.decode('utf-8').strip()}")


### Testing

A really simple function for testing that our solution produces the expected test output.

In [9]:
def validate(test, answer):
    """
    Args:
        test: the answer given by our solution
        answer: the expected answer, e.g. from instructions
    """
    if test != answer:
        raise AssertionError(f"{test} != {answer}")

### Useful Helper Classes

In [10]:
#################################################################
# POINTS, VECTORS AND GRIDS
#################################################################

@dataclass(frozen=True)
class Point:
    """ Class for storing a point x,y coordinate """
    x: int
    y: int

    def __add__(self, other: Point):
        return Point(self.x + other.x, self.y + other.y)

    def __mul__(self, other: Point):
        """ (x, y) * (a, b) = (xa, yb) """
        return Point(self.x * other.x, self.y * other.y)

    def __sub__(self, other: Point):
        return self + Point(-other.x, -other.y)

    def __lt__(self, other):
        # Arbitrary comparison logic
        return (self.x, self.y) < (other.x, other.y)
    
    def yield_neighbours(self, include_diagonals=True, include_self=False):
        """ Generator to yield neighbouring Points """

        deltas: list
        if not include_diagonals:
            deltas = [vector.value for vector in Vectors if abs(vector.value[0]) != abs(vector.value[1])]
        else:
            deltas = [vector.value for vector in Vectors]

        if include_self:
            deltas.append((0, 0))

        for delta in deltas:
            yield Point(self.x + delta[0], self.y + delta[1])

    def neighbours(self, include_diagonals=True, include_self=False) -> list[Point]:
        """ Return all the neighbours, with specified constraints.
        It wraps the generator with a list. """
        return list(self.yield_neighbours(include_diagonals, include_self))

    def get_specific_neighbours(self, directions: list[Vectors]) -> list[Point]:
        """ Get neighbours, given a specific list of allowed locations """
        return [(self + Point(*vector.value)) for vector in list(directions)]

    @staticmethod
    def manhattan_distance(a_point: Point) -> int:
        """ Return the Manhattan distance value of this vector """
        return sum(abs(coord) for coord in asdict(a_point).values())

    def manhattan_distance_from(self, other: Point) -> int:
        """ Manhattan distance between this Vector and another Vector """
        diff = self-other
        return Point.manhattan_distance(diff)

    def __repr__(self):
        return f"P({self.x},{self.y})"

class Vectors(Enum):
    """ Enumeration of 8 directions.
    Note: y axis increments in the North direction, i.e. N = (0, 1) """
    N = (0, 1)
    NE = (1, 1)
    E = (1, 0)
    SE = (1, -1)
    S = (0, -1)
    SW = (-1, -1)
    W = (-1, 0)
    NW = (-1, 1)

    @property
    def y_inverted(self):
        """ Return vector, but with y-axis inverted. I.e. N = (0, -1) """
        x, y = self.value
        return (x, -y)

class VectorDicts():
    """ Contains constants for Vectors """
    ARROWS = {
        '^': Vectors.N.value,
        '>': Vectors.E.value,
        'v': Vectors.S.value,
        '<': Vectors.W.value
    }

    DIRS = {
        'U': Vectors.N.value,
        'R': Vectors.E.value,
        'D': Vectors.S.value,
        'L': Vectors.W.value
    }

    NINE_BOX: dict[str, tuple[int, int]] = {
        # x, y vector for adjacent locations
        'tr': (1, 1),
        'mr': (1, 0),
        'br': (1, -1),
        'bm': (0, -1),
        'bl': (-1, -1),
        'ml': (-1, 0),
        'tl': (-1, 1),
        'tm': (0, 1)
    }

class Grid():
    """ 2D grid of point values. """
    def __init__(self, grid_array: list) -> None:
        self._array = grid_array
        self._width = len(self._array[0])
        self._height = len(self._array)

    def value_at_point(self, point: Point):
        """ The value at this point """
        return self._array[point.y][point.x]

    def set_value_at_point(self, point: Point, value):
        self._array[point.y][point.x] = value

    def valid_location(self, point: Point) -> bool:
        """ Check if a location is within the grid """
        if (0 <= point.x < self._width and  0 <= point.y < self._height):
            return True

        return False

    @property
    def width(self):
        """ Array width (cols) """
        return self._width

    @property
    def height(self):
        """ Array height (rows) """
        return self._height

    def all_points(self) -> list[Point]:
        points = [Point(x, y) for x in range(self.width) for y in range(self.height)]
        return points

    def rows_as_str(self):
        """ Return the grid """
        return ["".join(str(char) for char in row) for row in self._array]

    def cols_as_str(self):
        """ Render columns as str. Returns: list of str """
        cols_list = list(zip(*self._array))
        return ["".join(str(char) for char in col) for col in cols_list]

    def __repr__(self) -> str:
        return f"Grid(size={self.width}*{self.height})"

    def __str__(self) -> str:
        return "\n".join("".join(map(str, row)) for row in self._array)

### Useful Helper Functions

In [11]:
#################################################################
# CONSOLE STUFF
#################################################################

def cls():
    """ Clear console """
    os.system('cls' if os.name=='nt' else 'clear')

#################################################################
# USEFUL FUNCTIONS
#################################################################

def binary_search(target, low:int, high:int, func, *func_args, reverse_search=False):
    """ Generic binary search function that takes a target to find,
    low and high values to start with, and a function to run, plus its args.
    Implicitly returns None if the search is exceeded. """

    res = None  # just set it to something that isn't the target
    candidate = 0  # initialise; we'll set it to the mid point in a second

    while low < high:  # search exceeded
        candidate = int((low+high) // 2)  # pick mid-point of our low and high
        res = func(candidate, *func_args) # run our function, whatever it is
        logger.debug("%d -> %d", candidate, res)
        if res == target:
            return candidate  # solution found

        comp = operator.lt if not reverse_search else operator.gt
        if comp(res, target):
            low = candidate
        else:
            high = candidate

def merge_intervals(intervals: list[list]) -> list[list]:
    """ Takes intervals in the form [[a, b][c, d][d, e]...]
    Intervals can overlap.  Compresses to minimum number of non-overlapping intervals. """
    intervals.sort()
    stack = []
    stack.append(intervals[0])

    for interval in intervals[1:]:
        # Check for overlapping interval
        if stack[-1][0] <= interval[0] <= stack[-1][-1]:
            stack[-1][-1] = max(stack[-1][-1], interval[-1])
        else:
            stack.append(interval)

    return stack

@cache
def get_factors(num: int) -> set[int]:
    """ Gets the factors for a given number. Returns a set[int] of factors.
        # E.g. when num=8, factors will be 1, 2, 4, 8 """
    factors = set()

    # Iterate from 1 to sqrt of 8,
    # since a larger factor of num must be a multiple of a smaller factor already checked
    for i in range(1, int(num**0.5) + 1):  # e.g. with num=8, this is range(1, 3)
        if num % i == 0: # if it is a factor, then dividing num by it will yield no remainder
            factors.add(i)  # e.g. 1, 2
            factors.add(num//i)  # i.e. 8//1 = 8, 8//2 = 4

    return factors

def to_base_n(number: int, base: int):
    """ Convert any integer number into a base-n string representation of that number.
    E.g. to_base_n(38, 5) = 123

    Args:
        number (int): The number to convert
        base (int): The base to apply

    Returns:
        [str]: The string representation of the number
    """
    ret_str = ""
    curr_num = number
    while curr_num:
        ret_str = str(curr_num % base) + ret_str
        curr_num //= base

    return ret_str if number > 0 else "0"


### Generic Initialisation


In [12]:
FOLDER = "aoc"
YEAR = 2023
logger_identifier = "aoc" + str(YEAR)
logger = retrieve_console_logger(logger_identifier)
logger.setLevel(logging.DEBUG)

# Days

Here you'll find a template to build a solution for a given day, and then the solutions for all days in this event.

To copy the template day, select all the cells in the `Day n` template, add a new cell at the end, and then paste the cells there.

---
## Day 1: Trebuchet?!

In [ ]:
DAY = 1
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.INFO)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 1 Part 1

And we're off!!  Welcome to the first day of Advent of Code 2023!!

Today was a troublesome start for me.  My Internet was out.  (Thanks, Virgin Media.) So, after unsuccessful restarts of the router and home network, I switched over to mobile hotspot.

Part 1 is pretty trivial, as we've come to expect. You need to identify the first and last digits of each line of a string. Concatenating these two values gives you a two digit number, which the puzzle calls a _calibration value_. Then we just add them all together.

**My Solution**

- For each line, I simply loop through each char in the line, and use the `isdigit()` method to determine if it is a digit.
- Then repeat, but this time, looping from the end using the Python construct `[::-1]` which just means: start from the end, and then step with increments of `-1`. I.e. move backwards.
- Finally, concatenate the two digits (still as strings), to update a two digit number. Then convert it to an int.
- Store all these ints in a list.  And at the end, return the sum of the list.

In [ ]:
num_words = {"one": 1,
             "two": 2,
             "three": 3,
             "four": 4,
             "five": 5,
             "six": 6,
             "seven": 7,
             "eight": 8,
             "nine": 9
             }

In [ ]:
def solve(data, with_spelled_nums=False):
    calibration_vals = []
    for line in data:
        logger.debug(line)
        
        first_posn = 1e6        
        last_posn = -1
        first = last = ""

        for posn, char in enumerate(line): # read from start
            if char.isdigit():
                first_posn = posn
                first = char
                break
            
        for posn, char in enumerate(line[::-1]): # read from the end
            if char.isdigit():
                last_posn = len(line) - posn - 1 # remember, we're now counting from the end!!
                last = char
                break

        if with_spelled_nums:
            for num_word in num_words:
                posn = line.find(num_word)
                if 0 <= posn < first_posn:
                    first_posn = posn
                    first = str(num_words[num_word]) # map it back to int
            
                posn = line.rfind(num_word)
                if posn > last_posn:
                    last_posn = posn
                    last = str(num_words[num_word]) # map to the int
        
        calibration_vals.append(int(first + last))
    
    return sum(calibration_vals)  
        

In [ ]:
%%time
sample_inputs = [["1abc2", "pqr3stu8vwx", "a1b2c3d4e5f", "treb7uchetabcdef"]]
sample_answers = [142]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve(curr_input), curr_ans) # test with sample data

soln = solve(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 1 Part 2

For Day 1, this wasn't quite as trivial as I was expecting! Now we have to also find the positions of any "spelled" versions of the digits 0-9.

**My solution:**

- Create a `dict` to store the spelled versions of 1-9, and map them to their respective int values.
- Now, with each line, perform the same code as we did for Part 1 to find the first and last positions of the digit representation. 
  - But this time, store the positions found, as well as the values. I use the [`enumerate()`](https://aoc.just2good.co.uk/python/enumerate) to give me the current position of each char in my line.
  - Be really careful when storing the position when counting from the end.  This tripped me up for a couple of minutes!!  When we're looping through chars from the end, backwards, we want to store the position in the string, not the current enumeration value. 
- Then, run another loop that looks for each spelled number in our dict of spelled numbers.
  - To search for our current spelled number in our line from the start, using the `find()` method.
  - To search for our current spelled number in our line from the end, using the `rfind()` method.
  - Whenever we find a spelled number, check whether we found it at a position that is earlier / later (as required) than the digit we found before.
  - Whenever I find such a spelled number, I convert the int value in the dict to a string, so that I can concatenate the string values, just as we did before.

In [ ]:
%%time
sample_inputs = [["two1nine", 
                  "eightwothree", 
                  "abcone2threexyz", 
                  "xtwone3four", 
                  "4nineeightseven2", 
                  "zoneight234", 
                  "7pqrstsixteen"]]
sample_answers = [281]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve(curr_input, with_spelled_nums=True), curr_ans) # test with sample data

soln = solve(input_data, with_spelled_nums=True)
logger.info(f"Part 2 soln={soln}")

---
## Day 2: Cube Conundrum

In [ ]:
DAY = "2" # replace with actual number (without leading digit)
logger.setLevel(logging.DEBUG)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 2 Part 1

In each game, we have a bag containing some number of red, green and blue cubes.  The bag is samples several times per game. Our input data shows these random samples for each game. E.g.

```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```

**Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?**

**My solution:**

- I create a CubeSample [class](https://aoc.just2good.co.uk/python/classes) to store each sample, i.e. the number of r, g, b cubes.
- I create a Game class to store the game ID and all the samples for that game.
- I parse the input with [regex](https://aoc.just2good.co.uk/python/regex). My approach was:
  - Split the game line into the game part, and the samples part. Retrieving the game ID is trivial.
  - For the samples, use a regex that looks for "n colour", and use a regex `finditer()` to find all matches for this.
  - Create a [defaultdict](https://aoc.just2good.co.uk/python/defaultdict) that sets the initial values for r, g, b to 0.
  - Then iterate over the matches from `finditer()``, and update the r, g, b as required.
- Now I simply loop through each game. 
  - For each game, I loop through the samples. If any sample has more r, g, b than we're allowed, then this game is impossible.
  - Build up a list of the games that are possible. Then sum up the IDs with a [comprehension](https://aoc.just2good.co.uk/python/comprehensions).

In [ ]:
@dataclass
class CubeSample:
    """ A sample contains a number of red, blue, and green cubes """
    red: int=0
    blue: int=0
    green: int=0

@dataclass
class Game:
    """ A game has an ID, and a random number of samples """
    id: int
    samples: list[CubeSample]

def parse_input(data) -> list[Game]:
    game_pattern = re.compile(r"Game\s+(\d+)")
    cubes_pattern = re.compile(r"(\d+)\s*(\w+)") # E.g. "3 blue" 
    
    games = []
    for line in data:
        game_part, samples_part = line.split(":")
        game_id = int(game_pattern.findall(game_part)[0])
        samples = samples_part.split(";")
        
        cube_samples = []
        for sample in samples:
            matches = cubes_pattern.finditer(sample)
            cube_counts = {"red": 0, "green": 0, "blue": 0} # reset cube counts for each sample
            for match in matches:
                cube_count, cube_colour = match.groups()
                cube_counts[cube_colour] = int(cube_count)
            
            cube_samples.append(CubeSample(cube_counts["red"], cube_counts["blue"], cube_counts["green"]))
        
        games.append(Game(game_id, cube_samples))
        
    return games
      
def solve_part1(games: list[Game]):
    """ Return the sum of the IDs for games that are possible. """
    
    allowed_red = 12
    allowed_green = 13
    allowed_blue = 14
    
    possible_games = []
    for game in games:
        possible = True
        for game_sample in game.samples:
            if (game_sample.red > allowed_red
                    or game_sample.green > allowed_green
                    or game_sample.blue > allowed_blue):
                possible = False
            
        if possible:
            possible_games.append(game)
            
    return sum(game.id for game in possible_games) 
            

In [ ]:
%%time
sample_inputs = [["Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green",
                  "Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue",
                  "Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red",
                  "Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red",
                  "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"]
                ]
sample_answers = [8]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    sample_games = parse_input(curr_input)
    validate(solve_part1(sample_games), curr_ans) # test with sample data

games = parse_input(input_data)
soln = solve_part1(games)
logger.info(f"Part 1 soln={soln}")

### Day 2 Part 2

**For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?**

Here, we need to look at all the samples for a given game, and determine the largest number of cubes shown of each colour, across the samples.

**My solution:**

Fortunately, since we already have our list of Games, this is now trivial to do. Simply iterate through the games, and for each game, iterate over all the samples. For each sample, determine if the number of any of r, g, b is greater than the biggest number of which we've found so far.

Then, multiply the r, g, b to get the `power` of the game. Then sum up all the powers.

In [ ]:
def solve_part2(games: list[Game]):
    """ Return the sum of the powers of all the games """
    game_powers = []
    for game in games:
        max_blue = max_green = max_red = 0
        for game_sample in game.samples:
            max_blue = max(max_blue, game_sample.blue)
            max_green = max(max_green, game_sample.green)
            max_red = max(max_red, game_sample.red)
     
        # We're told that power = product of r, g, b   
        game_powers.append(max_blue*max_green*max_red)
    
    return sum(game_powers)    
        

In [ ]:
%%time
sample_inputs = [["Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green",
                  "Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue",
                  "Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red",
                  "Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red",
                  "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"]
                ]
sample_answers = [2286]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    sample_games = parse_input(curr_input)
    validate(solve_part2(sample_games), curr_ans) # test with sample data

soln = solve_part2(games)
logger.info(f"Part 2 soln={soln}")

---
## Day 3: Gear Ratios

In [ ]:
DAY = "3" # replace with actual number (without leading digit)
logger.setLevel(logging.DEBUG)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 3 Part 1

I'm finding AoC fairly tough this year. I wasn't expecting the early challenges to be this tricky.

Anyhoo...

We're given a 2D grid, called the _engine schematic_. That grid contains numbers, periods (which should be ignored), and symbols (anything else). We need to determine the _part numbers_, which are told are any numbers adjacent to a symbol.

**What is the sum of all of the part numbers in the engine schematic?**

**My solution:**

- I get to reuse one of my [helper classes](https://aoc.just2good.co.uk/python/reusable_code).  Yay!  
  - I'm going to reuse my `Point` class, which stores x, y coordinates, but also has the ability to return all of its adjacent neighbours.
  - I'm going to reuse my `Grid` class, which already knows how to create a 2D grid, iterate through the points in the grid, get the values at any location, and determine if a specied point is in the grid.
- I create a new [class](https://aoc.just2good.co.uk/python/classes) called `EngineGrid` by extending `Grid`.
  - This class knows how to return all the points that are symbols.
- To solve:
  - First, get all the symbol locations. This is trivial.
  - Get the value of this symbol, and store it alongside the location in a dictionary.
  - Then, get all the neighbour locations for each symbol location.
  - Check if this neighbour is part of a part number range we've already found.  If it is, then we don't need to check this neighbour; we already know it's part of a part number.
  - If the neighbour is a valid location, check if it is a digit. If it is, then this location is _in_ a part number. If so, use the method `get_part_number_continugous_range()` to determine the full set of points that make up this part number. It works by taking this location on this line of the grid, and walking backwards and fowards, until the value found is no longer a digit. We return the full set of contiguous digits as a part number range.
  - Add this to the [set](https://aoc.just2good.co.uk/python/sets) of ranges for this symbol, i.e. in the `ranges_for_symbol` dict. Note, here I'm using a [defaultdict](https://aoc.just2good.co.uk/python/defaultdict), such that I can initialise each entry with an empty `set`, and then add to the set whenever we find a range.
  - Also, add all the points from this range to a `set` called `all_range_locations_for_symbol`. We use this when checking each neighbour, to see if this neighbour is already part of a range associated with this symbol.
  - Also, determine the part number for this range, using our `get_part_number_for_range()` method. Store the result in a `dict` called `part_range_to_num`, where the key is the range `tuple` itself.
  - Finally, we can sum up all the part number values, by summing the values from the `dict` of `part_range_to_num`.

### Day 3 Part 2

Now we're told we need to find symbols that are _gears_, i.e. the symbols that are simply `*`. And we need to find all the gears that have exactly two adjacent part numbers. Where this is true, the product of the two part numbers is the _gear ratio_. Then we need to add up all the gear ratios.

**What is the sum of all of the gear ratios in your engine schematic?**

**My solution:**

This is a fairly trivial addition. We already have the `ranges_for_symbol` dictionary. So, in our loop that processes each symbol, we now:

- Check if the symbol is a gear, i.e. `*`
- If so, check how many ranges are associated with this symbol.
- If there are exactly two ranges, then we need to determine the part number values of this ranges, and multiply them together to obtain our `gear_ratio` for this gear.
- Finally, add up all the gear ratios.



In [ ]:
class EngineGrid(Grid):
    def get_symbol_locations(self) -> list[Point]:
        """ Return all locations that contain a symbol """
        symbol_locations = [point for point in self.all_points() if self._is_symbol(point)]
        return symbol_locations
    
    def _is_symbol(self, point: Point) -> bool:
        """ A symbol is anything that is not numeric, or not a period. """
        val = str(self.value_at_point(point))
        if val.isdigit():
            return False
        
        if val == ".":
            return False
        
        return True
    
    def get_part_number_contiguous_range(self, point: Point) -> tuple[Point, ...]:
        """ Given a point within a part number, we want to return the entire range or points that make up that part number. """
        line = self._array[point.y] # get the row this point is on
    
        # Find the start of the contiguous digits
        start = point.x
        while start > 0 and line[start - 1].isdigit():
            start -= 1

        # Find the end of the contiguous digits
        end = point.x
        while end < len(line) - 1 and line[end + 1].isdigit():
            end += 1

        # Return the contiguous locations that make up a part number
        contiguous_locations = [Point(x, point.y) for x in range(start, end+1)]
        return tuple(contiguous_locations)

    def get_part_number_for_range(self, part_range: tuple[Point, ...]) -> int:
        """ Given a set of points that make up a part number, return the part number they contain. """
        part_num = ""
        for point in part_range:
            part_num += self.value_at_point(point)
            
        return int(part_num)

In [ ]:
def solve(input_data) -> tuple[int, int]:
    """ Part 1: determine the sum of all part numbers, where a part number is 
                a full set of continguous digits adjacent to a symbol. 
        Part 2: determine the gear ratios, where a gear ratio is the product of the part numbers
                adjacent to a gear, for any gear that has exactly two adjacent part numbers. """
    points_and_symbols = [] # (point, value-for-point)
    part_number_ranges = set() # = all the points in a range. Use set so we don't double count ranges.
    ranges_for_symbol = defaultdict(set) # { (point, value), {ranges} }
    part_range_to_num = {} # so we can cache the part number corresponding to a range
    gear_ratios = [] # a gear ratio is given by the product of its two adjacent part numbers
    
    engine = EngineGrid(input_data)
    
    # get the locations of symbols, e.g. * ?, but not .
    symbol_locations = engine.get_symbol_locations()
    for point in symbol_locations:
        all_range_locations_for_symbol = set()
        symbol_val = engine.value_at_point(point)
        points_and_symbols.append((point, symbol_val))
        
        # get adjacent locations to this symbol
        for neighbour in point.neighbours():
            if neighbour in all_range_locations_for_symbol:
                continue    # we don't care about neighbours in ranges we've already found
            
            if engine.valid_location(neighbour): # check it is in the grid
                val = str(engine.value_at_point(neighbour))
                if val.isdigit(): # this neighbour is a point in a part number
                    range_for_locn = engine.get_part_number_contiguous_range(neighbour) # gets range as tuple
                    part_range_to_num[range_for_locn] = engine.get_part_number_for_range(range_for_locn)
                    ranges_for_symbol[(point, symbol_val)].add(range_for_locn) # add this range for this symbol
                    all_range_locations_for_symbol.update(set(range_for_locn)) # add all locations from this range
                
        if symbol_val == "*": # if this is a gear
            gear_ranges = ranges_for_symbol[(point, symbol_val)]
            if gear_ranges and len(gear_ranges) == 2: # if this gear has exactly two ranges
                gear_part_nums = [part_range_to_num[gear_range] for gear_range in gear_ranges]
                gear_ratios.append(reduce(operator.mul, gear_part_nums))
                
        part_number_ranges.update(ranges_for_symbol[(point, symbol_val)])
        
    return sum(part_range_to_num.values()), sum(gear_ratios)


In [ ]:
%%time
sample_inputs = [["467..114..",
                  "...*......",
                  "..35..633.",
                  "......#...",
                  "617*......",
                  ".....+.58.",
                  "..592.....",
                  "......755.",
                  "...$.*....",
                  ".664.598.."]]
sample_answers = [(4361, 467835)]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve(curr_input), curr_ans) # test with sample data

part_num_sum, gear_ratios_sum = solve(input_data)
logger.info(f"Part 1 soln: {part_num_sum=}")
logger.info(f"Part 2 soln: {gear_ratios_sum=}")

---
## Day 4: Scratchcards

In [13]:
DAY = "4" # replace with actual number (without leading digit)
logger.setLevel(logging.DEBUG)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

#### See [Day 4](https://adventofcode.com/2023/day/4).

In [14]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.INFO)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

14:53:13.177:aoc2023 - INF: Input data:
  1: Card   1: 20 72 30 38 18 65  6 55 70 27 | 12 28 47 50 60 17 14 25 41 95 66 88 61 52 76  5 23 77 31 32 99 89 53 54 96
  2: Card   2: 15 98 12  3 20 60 58 54 34 18 | 98 23 12 19 61 38 11 43 58 97 63 10 49 67 44 52 88  4 22  1 42 65 20 13 25
  3: Card   3: 47 97  2 80 89 56 66 85 62 46 | 78 97 47 14 15 85 42 66 24 28 54 46 89 62 80 56 22  5 57 52 69  2  3 95 19
  4: Card   4: 60 59 88 14 53 46 96 29 99 37 | 52 46 92 26 72 74 99 38  7 65 43 63 39 36 44  9 56 42 79 50 89 48 85 28 27
  5: Card   5: 44 46 42  9 65 98 97 67 72 22 | 80 14 29 39 98 64  9 46 52 24 69 22 51 65 66 20 72 21 55 12 97 42 44 41 67
...
183: Card 183: 58 73 31 77 33 10 49 92 28 25 | 39  3 96 41 45 32 82 79  8 21 36  9  6 17 84 14 27 20 88 60 98 43 61 24  5
184: Card 184: 10 41 56 32 84 29 85 26 34 27 |  6  4  9 38 25 26  8 86 68 87 11 30  5 79 78 52 49 81 65 64 40  2 37 39 17
185: Card 185: 95 13 15 61 49 19 60 21 33 87 | 30 36 52 88 23 65 95 25 53 51 77 32 66 31 68 84 39 29  

### Day 4 Part 1

We have a set of scratch cards.  Each card has an `id`, then a set of `winning` numbers, then a set of `actual` numbers that were scratched. The score of each scratch card depends on the number of winning numbers we have matched. The first match gives us 1 point, and each additional match doubles the score.

**How points are the pile of scratch cards worth, in total?**

**My solution:**

This is pretty simple.

- I create a `ScratchCard` [dataclass](https://aoc.just2good.co.uk/python/classes#dataclass) which:
  - Stores the card ID.
  - Stores the winning numbers, and the scratched numbers, both as sets.  I want to use sets because I will want to determine the intersection of these two sets, i.e. how many winning numbers were matched. [Set algebra](https://aoc.just2good.co.uk/python/sets) makes this super easy.
  - Has a `score()` method which determines the score, according to the rules.
- Then I parse the input data using [regex](https://aoc.just2good.co.uk/python/regex). Let me explain how \
  `Card(?:\s+)(\d+):\s*((?:\d+\s*)+)(?: \|\s+)((?:\d+\s*)+)` works:
  - `(\d+)`: This is a capturing group that matches one or more digits. The `\d` is a shorthand character class that matches any digit (0-9), and the `+` means one or more of the preceding element. This is how I capture the `id`.
  - `\s*`: This matches zero or more whitespace characters. The `*` means zero or more of the preceding element.
  - `((?:\d+\s*)+)`: This is a capturing group containing a non-capturing group. The non-capturing group `(?:\d+\s*)` matches one or more digits followed by zero or more whitespace characters. The outer capturing group with the `+` at the end repeats this pattern one or more times. This is how I capture ALL of the `winning` numbers.
  - `(?: \|\s+)`: This is another non-capturing group. It matches a literal space, followed by a vertical bar `|`, followed by one or more whitespace characters.
  - `((?:\d+\s*)+)`: The same as before. This is how I capture ALL of the `actual` numbers.
- I use this regex to parse each line:
  - This gives me the `id`, a string containing all the `winning` numbers, and a string containing all the `actual` numbers.
  - I split the `winning` and `actual` numbers at the space, using `split()`. This gives me a list of string values. Then I convert the string values to `int` using `map()`, and finally convert each `list` to a `set`.
  - Now I can create an instance of `ScoreCard` from the `id` and the two sets.

- Finally, just add up all the scores from each card using `sum` and a [list comprehension](https://aoc.just2good.co.uk/python/comprehensions).

In [15]:
@dataclass
class ScratchCard:
    """ A scratchcard has an ID, winning numbers, and (actual) numbers scratched. """
    id: int
    winning: set[int]
    actual: set[int]
    
    def matches(self) -> int:
        """ Return the number of winning numbers we have matched. """
        return len(self.winning & self.actual)
    
    def score(self) -> int:
        """ For every winning number, double the score. Examples scores...
        0 matches -> 0
        1 match   -> 1
        2 matches -> 2
        3 matches -> 4, etc """
        num_matches = self.matches()
        return 2**(num_matches-1) if num_matches > 0 else 0

In [16]:
def parse_cards(data) -> list[ScratchCard]:
    scratch_card_pattern = r"Card(?:\s+)(\d+):\s*((?:\d+\s*)+)(?: \|\s+)((?:\d+\s*)+)"
    scratch_card_matcher = re.compile(scratch_card_pattern)
    scratch_cards = []
    for line in data:
        id, winning, actual = scratch_card_matcher.findall(line)[0]
        
        id = int(id)
        winning = set(map(int, winning.split()))
        actual = set(map(int, actual.split()))
        scratch_cards.append(ScratchCard(id, winning, actual))
        
    return scratch_cards

In [17]:
def solve_part1(cards: list[ScratchCard]):  
    return sum(card.score() for card in cards)    

In [18]:
%%time
sample_inputs = [["Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53",
                  "Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19",
                  "Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1",
                  "Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83",
                  "Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36",
                  "Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"]
]
sample_answers = [13]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    sample_cards = parse_cards(curr_input)  
    validate(solve_part1(sample_cards), curr_ans) # test with sample data

cards = parse_cards(input_data)
soln = solve_part1(cards)
logger.info(f"Part 1 soln={soln}")

14:53:25.271:aoc2023 - INF: Part 1 soln=22674


CPU times: total: 0 ns
Wall time: 3 ms


### Day 4 Part 2

Now we're told that there's no such thing as a _score_. Instead scratchcards only cause you to win more scratchcards equal to the number of winning numbers you have. You win copies of the scratchcards below the winning card equal to the number of matches.

**how many total scratchcards do you end up with?**

**My solution**

I spent several minutes thinking about how to solve this problem, before writing any code.  At first, I was thinking about some sort of `while loop` that iterates through cards, and only exits when there are no more cards. I was thinking I could insert duplicate cards into the list as I iterate.

But then I realised that I didn't need to do that.  I simply needed to keep a count of how many there are of each card ID. I can do that with a `dict`. So my strategy:

- Create a `dict` that stores the number of cards for each card ID.
- Loop through all the cards we have to initialise the dict. There will be one of each.
- Then, loop through the cards again, in order.  For each card:
  - Get the number of matches.
  - Use this number of matches to determine the successive cards that need to be duplicated.
  - Increment the count of each of those successive cards, by the count of the card we're currently on. (Because it might not be 1.)

Even though it took my tiny brain a little while to realise this approach, the actual code is pretty trivial.

The solution is fast, and only takes a couple of milliseconds.

In [19]:
def solve_part2(cards: list[ScratchCard]):
    card_counts = {}
    for card in cards: # initialise our card counts 
        card_counts[card.id] = 1
    
    for card in cards:
        count_this_card = card_counts[card.id]
        for i in range(card.id+1, card.id+1+card.matches()):
            card_counts[i] += count_this_card
    
    logger.debug(card_counts)
    total_card_count = sum(card_counts.values())
    return total_card_count

In [20]:
%%time
sample_inputs = [["Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53",
                  "Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19",
                  "Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1",
                  "Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83",
                  "Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36",
                  "Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"]
]
sample_answers = [30]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    sample_cards = parse_cards(curr_input)  
    validate(solve_part2(sample_cards), curr_ans) # test with sample data

soln = solve_part2(cards)
logger.info(f"Part 2 soln={soln}")

14:53:31.776:aoc2023 - INF: Part 2 soln=5747443


CPU times: total: 0 ns
Wall time: 1 ms


---
## Day 5: If You Give A Seed A Fertilizer

In [13]:
DAY = "5" # replace with actual number (without leading digit)
logger.setLevel(logging.DEBUG)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

#### See [Day 5](https://adventofcode.com/2023/day/5).

In [38]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.INFO)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

22:52:29.042:aoc2023 - INF: Input data:
  1: seeds: 1778931867 1436999653 3684516104 2759374 1192793053 358764985 1698790056 76369598 3733854793 214008036 4054174000 171202266 3630057255 25954395 798587440 316327323 290129780 7039123 3334326492 246125391
  2: 
  3: seed-to-soil map:
  4: 1965922922 2387203602 59808406
  5: 2540447436 434094583 220346698
...
231: 3584396646 1757986922 143559505
232: 267143672 161914533 100367854
233: 3527298136 3080301496 57098510
234: 1082510516 3749523110 18833794
235: 3727956151 3150282471 536984712


### Day 5 Part 1

Our input includes:

- Seeds to be planted, as ints.
- Maps, in the the format: 
  - destination-range-start, source-range-start, range-length. \
    The source and destination ranges will be the same length.
- We have the following maps:
  - seed-to-soil
  - soil-to-fertilizer
  - fertilizer-to-water
  - water-to-light
  - light-to-temperature
  - temperature-to-humidity
  - humidity-to-location

E.g. 

```
soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15
```

Unmapped source numbers map to the same destination number.

**What is the lowest location number that corresponds to any of the initial seed numbers?**

**My solution:**

- We need a rules engine that takes a value and determines the target value for that map.
- I've done this by creating a class called `GardinerMap`.
- Each instance of `GardinerMap` contains the source and target ranges for that map type.
- The `GardinerMap`:
  - Stores all source ranges in that map as a list of tuples, with each tuple being `(source start, range length)`.
  - Stores all destination ranges in that map as a list of tuples, with each tuple being `(destination start, range length)`.
  - All the actual mapping is done in the method `get_target()`. It iterates through each range, and determines if our input value is part of that range. If it is, we apply the required _shift_, which is the difference between the destination range start and the source range start. Now we've finished mapping, so we return the mapped value.

We need to start by passing the seed value to `get_target()` of our first `GardinerMap`. This returns a new value.  We then take this value and pass it into `get_target()` of the second `GardinerMap`.  And so on, until we've done this for every `GardinerMap`.



  - Finally, back in our `solve_part1()` method.


In [45]:
@dataclass
class GardinerMap():
    source_type: str
    dest_type: str
    
    src_ranges: list[tuple[int,int]] = field(default_factory=list) # [(src_start, length), (src_start, length), ... ]
    dest_ranges: list[tuple[int,int]] = field(default_factory=list) # [(dest_start, length), (dest_start, length), ... ]
    
    def add_range(self, src_start: int, dest_start: int, range_length: int):
        """ Add a range which contains src start, dest start, and the length of the range """
        self.src_ranges.append((src_start, range_length)) 
        self.dest_ranges.append((dest_start, range_length))
        
    def _sort_ranges(self):
        """ Sort the range into ascending numeric, based on source range start values """
        
        # Sort src_ranges and get the order of indices
        index_order = sorted(range(len(self.src_ranges)), key=lambda i: self.src_ranges[i][0])

        # Now sort both ranges
        self.src_ranges = [self.src_ranges[i] for i in index_order]
        self.dest_ranges = [self.dest_ranges[i] for i in index_order]

    def finalise(self):
        """ Sort the range into ascending numeric, based on source range start values """
        self._sort_ranges()
        
    def get_target(self, src_val: int):
        """ Map a source value to a target value """
        target = src_val # if our source isn't in a range, then return the same value
        
        for i, curr_range in enumerate(self.src_ranges):
            src_start = curr_range[0]
            src_end = curr_range[0] + curr_range[1] # exclusive end
            if src_start <= src_val < src_end: # if our source is in a range, then apply the shift
                target = src_val - src_start + self.dest_ranges[i][0]
                break # we've mapped the value, so no more ranges need to be checked
        
        return target

    # For Part 2
    def map_intervals(self, src_intervals: list[tuple[int, int]]) -> list[tuple[int, int]]:
        """ 
        Take input ranges and return output ranges.
        - src_ranges: [(rng1_start, rng1_end), (rng2_start, rng2_end), ... ] 
        """
        new_intervals = []
        
        # Iterate through the ranges, just as we did when mapping a single seed
        for i, curr_range in enumerate(self.src_ranges): 
            src_start = curr_range[0]
            src_end = curr_range[0] + curr_range[1] # exclusive end
            dest = self.dest_ranges[i][0]

            temp_intervals = []

            while src_intervals: # process the current interval       
                (int_start, int_end) = src_intervals.pop()
                
                # Split the interval using the ranges in our map
                #### Scenario 1: ####
                # [int_start                                  int_end]
                #            [src_start      src_end]
                # [left     ][mid                   ][right          ]
                #
                #### Scenario 2: ####
                #                [int_start         int_end]
                #   [src_start     src_end]
                #   [n/a        ][mid     ][right          ]
                left = (int_start, min(int_end, src_start))
                mid = (max(int_start, src_start), min(src_end, int_end))
                right = (max(src_end, int_start), int_end)
                
                if left[1] > left[0]: # if left has +ve length, then scenario 1, else scenario 2
                    temp_intervals.append(left) # pass on the interval unchanged
                if mid[1] > mid[0]: # if mid has +ve length, then we need to apply the shift to this interval
                    # furthermore, once mapped, we know this interval wont appear in another range
                    new_intervals.append((mid[0]-src_start+dest, mid[1]-src_start+dest))
                if right[1] > right[0]:  # if right has +ve length
                    temp_intervals.append(right) # pass on the interval unchanged
            
            src_intervals = temp_intervals
                    
        return new_intervals + src_intervals

In [46]:
def parse_data(data: list[str]) -> tuple[list[int], list[GardinerMap]]:
    """ Parse input data, and convert to:
    - seeds: list of int
    - GardinerMap instances: list of GardinerMap """
    
    seeds = []
    source_maps = [] # Store our GardinerMap instances
    
    # Process the input file line by line, and switch modes based on the line last read
    current_map = None 
    for line in data:
        if line.startswith("seeds"): # The first line contains the seeds values
            _, seeds_part = line.split(":")
            seeds = [int(x) for x in seeds_part.split()]
        elif "map:" in line: # start of a map block; enter map processing mode
            map_src, _, map_dest = line.split()[0].split("-")
            current_map = GardinerMap(map_src, map_dest) # initialise our GardinerMap
        elif not line: # empty line, so finish with the current_map and add it to our list of GardinerMaps
            if current_map:
                current_map.finalise()
                source_maps.append(current_map)  # add it to the list
                current_map = None # and ensure we're no longer in map processing mode
        else: # we're in map processing mode
            assert line[0].isdigit(), "Line must start with numbers"
            assert current_map, "We must be adding to a Map now"
            dest_start, src_start, interval_len = [int(x) for x in line.split()]
            current_map.add_range(src_start=src_start, dest_start=dest_start, range_length=interval_len)
    
    # We don't read a final empty line at the end of the input, but I still want to finalise the block
    if current_map: # process the final map
        current_map.finalise()
        source_maps.append(current_map)   # add it to the dict
            
    return seeds, source_maps

In [47]:
def solve_part1(data):
    seeds, source_maps = parse_data(data)
    location_map = {}
    
    logger.debug(f"{seeds=}")
    for current_map in source_maps:
        logger.debug(f"{current_map=}")
        
    for seed in seeds:
        current_val = seed
        for current_map in source_maps:
            current_val = current_map.get_target(current_val)
        
        location_map[seed] = current_val
    
    logger.debug(f"Seeds->locations: {location_map}")
    return min(location_map.values())

In [48]:
%%time
sample_inputs = ["""seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""]
sample_answers = [35]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

23:06:00.877:aoc2023 - INF: Part 1 soln=107430936


CPU times: total: 0 ns
Wall time: 7.01 ms


---
### Day 5 Part 2

_"The values on the initial `seeds:` line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range."_

Uh oh, the values on the seeds line now describes ranges of seed numbers. In the real input, these numbers are huge. We're going to have way too many seeds to iterate over each seed.

Instead of mapping a source seed to a target value, we need to map the entire interval. By the way... **I hate intervals.**

I was struggling to get my head around this problem, so I created a sketch:

![Fertiliser range mappings](https://aoc.just2good.co.uk/assets/images/fertiliser.png)

Things to note:

- We start with intervals that represent our seeds.
- In each map, any given target range will always be the same length as the source range.
- As we propagate our seed intervals through each map, the following rules will apply:
  - If the interval sits outside a mapped range, the target interval will be the same.
  - If the interval sits inside a mapped range, the target interval will be shifted by the appropriate delta.
  - If the interval is both within and outside a mapped range, then we will need to split the interval. The part that is in the range will be moved by the appropriate delta.  That part that is out of the range will stay the same.
  - An interval might span multiple ranges. In this scenario, the interval must be split accordingly which will create more intervals.
  - Although it is possible to get intervals that are adjacent, a given source interval will never create overlapping intervals. Phew!

**My solution:**

- We can't map individual seed values, as there's just too many. Instead, we can map the intervals of seed values, and we only care about any boundaries. I.e. at the beginning of end of any mapping range, at the beginning and end of any seeds interval, and at any intersections.
- I added a `map_intervals()` method to my `GardinerMap` class. The job of this method is to map the supplied intervals into a new set of intervals. It works like this:
  - Take all the intervals supplied (which will initally be the seed intervals) as an input list.
  - Then iterate through our mapping ranges, as we did before.
  - For each range, determine the source start, source end, and destination start values, as we did before.
  - Initialise `temp_intervals`. We use this to store intervals that we still need to map.
  - Now, loop over each interval supplied as input. With each loop iteration, `pop`` the current interval off the list.
  - For each interval, determine the boundary conditions that make up the left, middle, and right parts of the interval, after intersecting with the current range. There may be no intersection, in which case only the `left` or `right` portion will be available.
  - If we have a `left` or `right` portion, then these are not intersecting with the current range.  So add these to our `temp_intervals`, and we can test them with the next range.
  - If we have a `mid` portion, then this portion is intersecting with the current range. If so, we need to _shift_ the range to obtain the new range, using exactly the same logic as we used for shifting individual values in Part 1. Note: once we've a interval portion like this, that portion will not be mapped again by this GardinerMap.
  - Now, at the end of the loop for this interval, continue to the next loop iteration, which will process the next interval with this range.
  - Once we've processed all intervals for this range, re-populate `src_intervals` using the current intervals stored in `temp_intervals`. This allows us to test these intervals with the next range.
  - Any intervals that are never mapped with a range will instead be passed on, as is.

Back in our `solve_part2()` method, we iterate over all the `GardinerMap` instances, just as we did with Part 1. But this time, we're passing in the seed intervals into the `map_intervals` method of the first `GardinerMap`. This returns a new list of intervals, which we pass into the next `GardinerMap`. And so on.

Eventually, we've left with the intervals generated by the _humidity-to-location_ map. This is a list of tuples, with each tuple containin the start and end values for that interval. So it's a trival matter to get the `min` of all the `start` values.

And that's it!

**That was a brutal day 5!!**

In [43]:
def solve_part2(data):
    location_map = {}
    seeds, source_maps = parse_data(data)
    
    # convert seeds to intervals, of format [(start, end), ...] where end is exclusive
    # let's call them intervals, to avoid confusion with the ranges we stored in our GardinerMap
    seed_intervals = [(seeds[i], seeds[i]+seeds[i+1]) for i in range(0, len(seeds), 2)]
    logger.debug(f"{seed_intervals=}")
    
    current_intervals = seed_intervals
    for current_map in source_maps:
        current_intervals = current_map.map_intervals(current_intervals)
        logger.debug(f"Mapping to {current_map.dest_type}: {current_intervals}")
        
    return min(start for start, _ in current_intervals)

In [44]:
%%time
sample_answers = [46]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

22:52:44.188:aoc2023 - INF: Part 2 soln=23738616


CPU times: total: 15.6 ms
Wall time: 20 ms


---
## Day n: title

In [ ]:
DAY = "n" # replace with actual number (without leading digit)
logger.setLevel(logging.DEBUG)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day n Part 1

Overview...

In [ ]:
def solve_part1(data):
    pass

In [ ]:
%%time
sample_inputs = ["abcdef"]
sample_answers = ["uvwxyz"]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input), curr_ans) # test with sample data

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day n Part 2

Overview...

In [ ]:
def solve_part2(data):
    pass

In [ ]:
%%time
sample_inputs = ["abcdef"]
sample_answers = ["uvwxyz"]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input), curr_ans) # test with sample data

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")